<a href="https://colab.research.google.com/github/nicolesuet/wnn-binarization/blob/main/wnn_binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.nn.functional import cross_entropy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch_dwn as dwn
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torchhd import embeddings

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
iris = fetch_ucirepo(id=53)
X = iris.data.features
y = iris.data.targets

In [ ]:
min_global = np.array(X.values).flatten().min()
max_global = np.array(X.values).flatten().max()

print(f"Min global: {min_global}")
print(f"Max global: {max_global}")

Min global: 0.1
Max global: 7.9


In [ ]:
encoder = LabelEncoder()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train = encoder.fit_transform(y_train.values.ravel())
y_test = encoder.fit_transform(y_test.values.ravel())
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

In [ ]:
torch_tensor = torch.tensor(X.values)
print(torch_tensor[:5])

tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000]], dtype=torch.float64)


In [ ]:
encoders = [
    {"encoding": "DISTRIBUTIVE", "encoder": dwn.DistributiveThermometer(10).fit(torch_tensor)},
    {"encoding": "GAUSSIAN", "encoder": dwn.GaussianThermometer(10).fit(torch_tensor)},
]

ADDRESS_SIZE = 10
IGNORE_ZERO = False
VERBOSE = False

In [ ]:
def evaluate(model, x_test, y_test):
    model.eval()
    with torch.no_grad():
        pred = (model(x_test.cuda(device)).cpu()).argmax(dim=1).numpy()
        acc = (pred == y_test.numpy()).sum() / y_test.shape[0]
    return acc

In [ ]:
for elem in encoders:
    print(f"\nEncoding: {elem['encoding']}\n")

    encoder = elem["encoder"]

    x_train = encoder.binarize(torch.tensor(X_train.values)).flatten(start_dim=1)
    x_test = encoder.binarize(torch.tensor(X_test.values)).flatten(start_dim=1)
    X_bin = encoder.binarize(torch.tensor(X.values)).flatten(start_dim=1)


    model = nn.Sequential(
      dwn.LUTLayer(x_train.size(1), 2000, n=6, mapping='learnable'),
      dwn.LUTLayer(2000, 1000, n=6),
      dwn.GroupSum(k=10, tau=1/0.3)
    )

    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=14)

    epochs = 1
    n_samples = x_train.shape[0]
    batch_size = 32


    for epoch in range(epochs):

      print(f"Epoch: {epoch}")

      model.train()

      permutation = torch.randperm(n_samples)
      correct_train = 0
      total_train = 0

      for i in range(0, n_samples, batch_size):
          optimizer.zero_grad()

          indices = permutation[i:i+batch_size]
          batch_x, batch_y = x_train[indices].cuda(device), y_train_tensor[indices].cuda(device)
          outputs = model(batch_x)
          loss = cross_entropy(outputs, batch_y)
          loss.backward()
          optimizer.step()

          pred_train = outputs.argmax(dim=1)
          correct_train += (pred_train == batch_y).sum().item()
          total_train += batch_y.size(0)

      train_acc = correct_train / total_train
      scheduler.step()
      test_acc = evaluate(model, x_test, y_test_tensor)
      print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')




Encoding: DISTRIBUTIVE

Epoch: 0
Epoch 1/1, Train Loss: 0.1461, Train Accuracy: 0.5900, Test Accuracy: 0.9000

Encoding: GAUSSIAN

Epoch: 0
Epoch 1/1, Train Loss: 0.3758, Train Accuracy: 0.5200, Test Accuracy: 0.9400


In [ ]:
    print(f"\nEncoding: SCATTER\n")

    emb = embeddings.Level(10, 20, "BSC", low=min_global, high=max_global, dtype=torch.uint8)

    x_train = emb(torch.tensor(X_train.values)).flatten(start_dim=1).float()
    x_test = emb(torch.tensor(X_test.values)).flatten(start_dim=1).float()
    X_bin = emb(torch.tensor(X.values)).flatten(start_dim=1).float()

    model = nn.Sequential(
      dwn.LUTLayer(x_train.size(1), 2000, n=6, mapping='learnable'),
      dwn.LUTLayer(2000, 1000, n=6),
      dwn.GroupSum(k=10, tau=1/0.3)
    )

    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=14)

    epochs = 1
    n_samples = x_train.shape[0]
    batch_size = 32


    for epoch in range(epochs):

      print(f"Epoch: {epoch}")

      model.train()

      permutation = torch.randperm(n_samples)
      correct_train = 0
      total_train = 0

      for i in range(0, n_samples, batch_size):
          optimizer.zero_grad()

          indices = permutation[i:i+batch_size]
          batch_x, batch_y = x_train[indices].cuda(device), y_train_tensor[indices].cuda(device)
          outputs = model(batch_x)
          loss = cross_entropy(outputs, batch_y)
          loss.backward()
          optimizer.step()

          pred_train = outputs.argmax(dim=1)
          correct_train += (pred_train == batch_y).sum().item()
          total_train += batch_y.size(0)

      train_acc = correct_train / total_train
      scheduler.step()
      test_acc = evaluate(model, x_test, y_test_tensor)
      print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')



Encoding: SCATTER

Epoch: 0
Epoch 1/1, Train Loss: 0.5944, Train Accuracy: 0.5000, Test Accuracy: 0.9000
